In [1]:
import numpy as np
import qcodes as qc

from qcodes.dataset import (
    LinSweep,
    Measurement,
    dond,
    experiments,
    initialise_or_create_database_at,
    load_by_run_spec,
    load_or_create_experiment,
    plot_dataset,
    load_by_id
)
from qcodes.instrument_drivers.mock_instruments import (
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)

In [ ]:
qubit_run_ids = {
    'Q1': 2729,
    'Q2': 2731,
    'Q3': 1831,
    'Q4': 1834,
    'Q5': 1844,
    'Q6': 1841,
    'Q7': 2738,
    'Q8': 2722
}

In [2]:
dac = DummyInstrument("dac", gates=["ch1", "ch2"])
dmm = DummyInstrumentWithMeasurement("dmm", setter_instr=dac)

In [3]:
initialise_or_create_database_at("arbok_inspector/test.db")

In [4]:
tutorial_exp = load_or_create_experiment(
    experiment_name="tutorial_exp", sample_name="synthetic data"
)
context_meas = Measurement(exp=tutorial_exp,  name="context_example")

In [44]:
context_meas.write_period = 2

context_meas.register_parameter(dac.ch1)
context_meas.register_parameter(dac.ch2)
context_meas.register_parameter(dmm.v1, setpoints=(dac.ch1, dac.ch2))

with context_meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 10):
        dac.ch1.set(set_v)
        set_v2 = set_v + 25
        dac.ch1.set(set_v+25)
        get_v = dmm.v1.get()
        datasaver.add_result( (dac.ch1, set_v), (dac.ch2, set_v +1), (dmm.v1, get_v))

    dataset = datasaver.dataset
    dataset.add_metadata('parallel_sweep_axes', str({"0": ["dac_ch1", "dac_ch2"]}))

Starting experimental run with id: 14. 


In [46]:
type(dataset)

qcodes.dataset.data_set.DataSet

In [35]:
dataset.run_id

13

In [ ]:
from plotly.codege

In [36]:
sweeps = dataset.metadata['parallel_sweep_axes'].replace("'", '"')
sweeps

'{"0": ["dac_ch1", "dac_ch2"]}'

In [32]:
import json
s = json.loads(sweeps)

In [33]:
s

{'0': 'dac_ch2', '1': 'dac_ch1'}

In [96]:
xdata = dataset.to_xarray_dataset()

['multi_index', 'dac_ch1', 'dac_ch2']

In [15]:
type(list(xdata.coords)[1])

str

In [65]:
xdata.coords['dac_ch1'].name

'dac_ch1'

In [101]:
for i in xdata.data_vars:
    print(i)

dmm_v1


In [102]:
next(iter(xdata.data_vars))

'dmm_v1'

In [104]:
import ast
import re

def parse_string_structure(s: str):
    # Quote barewords that are not already quoted or numbers
    s_quoted = re.sub(r'\b([a-zA-Z_][a-zA-Z0-9_]*)\b', r'"\1"', s)
    return ast.literal_eval(s_quoted)

# Example
input_str = "[(a, b), c]"
result = parse_string_structure(input_str)
print(result)  # [('a', 'b'), 'c']


[('a', 'b'), 'c']


In [105]:
input_str = "p1p2"
result = parse_string_structure(input_str)
print(result)

p1p2


In [41]:
dmm.v1.depends_on

AttributeError: 'DmmExponentialParameter' object has no attribute 'depends_on'

In [16]:
dims = [c for c in xdata.coords if c != 'multi_index']
test_dict = dict(zip(range(len(dims)), dims))
test_dict
dataset.add_metadata('sweep_dict', test_dict)

Rolling back due to unhandled exception
Traceback (most recent call last):
  File "C:\Users\z5440260\GitRepos\arbok_inspector\.venv\Lib\site-packages\qcodes\dataset\sqlite\connection.py", line 161, in atomic
    yield conn
  File "C:\Users\z5440260\GitRepos\arbok_inspector\.venv\Lib\site-packages\qcodes\dataset\sqlite\connection.py", line 215, in atomic_transaction
    c = transaction(atomic_conn, sql, *args)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\z5440260\GitRepos\arbok_inspector\.venv\Lib\site-packages\qcodes\dataset\sqlite\connection.py", line 191, in transaction
    c.execute(sql, args)
sqlite3.ProgrammingError: Error binding parameter 1: type 'dict' is not supported
Rolling back due to unhandled exception
Traceback (most recent call last):
  File "C:\Users\z5440260\GitRepos\arbok_inspector\.venv\Lib\site-packages\qcodes\dataset\sqlite\connection.py", line 161, in atomic
    yield conn
  File "C:\Users\z5440260\GitRepos\arbok_inspector\.venv\Lib\site-packages

RuntimeError: Rolling back due to unhandled exception

In [79]:
dataset.metadata

{'sweep_dict': "{1: 'dac_ch2', 2: 'dac_ch1'}"}

In [55]:
xdata = dataset.to_xarray_dataset()['dmm_v1']
xdata

<xarray.DataArray 'dmm_v1' (multi_index: 10)> Size: 80B
array([ 5.10954772,  2.80677339,  1.68680856,  0.99796751,  0.55273696,
        0.40738455,  0.02769668,  0.19296488,  0.0388273 , -0.06637338])
Coordinates:
  * multi_index  (multi_index) object 80B MultiIndex
  * dac_ch2      (multi_index) float64 80B 0.0 2.778 5.556 ... 19.44 22.22 25.0
  * dac_ch1      (multi_index) float64 80B 0.0 2.778 5.556 ... 19.44 22.22 25.0
Attributes:
    name:           dmm_v1
    paramtype:      numeric
    label:          Gate v1
    unit:           V
    inferred_from:  []
    depends_on:     ['dac_ch2', 'dac_ch1']
    units:          V
    long_name:      Gate v1

In [56]:
xdata.coords

Coordinates:
  * multi_index  (multi_index) object 80B MultiIndex
  * dac_ch2      (multi_index) float64 80B 0.0 2.778 5.556 ... 19.44 22.22 25.0
  * dac_ch1      (multi_index) float64 80B 0.0 2.778 5.556 ... 19.44 22.22 25.0

In [59]:
xdata.sel({'multi_index': 0})

<xarray.DataArray 'dmm_v1' (dac_ch1: 1)> Size: 8B
array([5.10954772])
Coordinates:
  * dac_ch1  (dac_ch1) float64 8B 0.0
    dac_ch2  int64 8B 0
Attributes:
    name:           dmm_v1
    paramtype:      numeric
    label:          Gate v1
    unit:           V
    inferred_from:  []
    depends_on:     ['dac_ch2', 'dac_ch1']
    units:          V
    long_name:      Gate v1

In [32]:
load_by_id(1).to_xarray_dataset()

<xarray.Dataset> Size: 1MB
Dimensions:                                    (qm_driver_iteration: 5,
                                                stability_map__vLevel_J1: 61,
                                                stability_map__vLevel_P1: 61)
Coordinates:
  * qm_driver_iteration                        (qm_driver_iteration) float64 40B ...
  * stability_map__vLevel_J1                   (stability_map__vLevel_J1) float64 488B ...
  * stability_map__vLevel_P1                   (stability_map__vLevel_P1) float64 488B ...
Data variables:
    stability_map__p1p2__chop                  (qm_driver_iteration, stability_map__vLevel_J1, stability_map__vLevel_P1) float64 149kB ...
    stability_map__p1p2__chop_state            (qm_driver_iteration, stability_map__vLevel_J1, stability_map__vLevel_P1) float64 149kB ...
    stability_map__p1p2__set1_feedback_adjust  (qm_driver_iteration, stability_map__vLevel_J1, stability_map__vLevel_P1) float64 149kB ...
    stability_map__p1p2__set1_feedback_error   (qm_driver_iteration, stability_map__vLevel_J1, stability_map__vLevel_P1) float64 149kB ...
    stability_map__p7p8__chop                  (qm_driver_iteration, stability_map__vLevel_J1, stability_map__vLevel_P1) float64 149kB ...
    stability_map__p7p8__chop_state            (qm_driver_iteration, stability_map__vLevel_J1, stability_map__vLevel_P1) float64 149kB ...
    stability_map__p7p8__set2_feedback_adjust  (qm_driver_iteration, stability_map__vLevel_J1, stability_map__vLevel_P1) float64 149kB ...
    stability_map__p7p8__set2_feedback_error   (qm_driver_iteration, stability_map__vLevel_J1, stability_map__vLevel_P1) float64 149kB ...
Attributes: (12/14)
    ds_name:                  p1p2, p7p8
    sample_name:              nova
    exp_name:                 stab_map
    snapshot:                 {"station": {"instruments": {"qdac": {"function...
    guid:                     571c4356-0000-0000-0000-019131d93552
    run_timestamp:            2024-08-08 11:56:44
    ...                       ...
    captured_counter:         1
    run_id:                   1
    run_description:          {"version": 3, "interdependencies": {"paramspec...
    parent_dataset_links:     []
    run_timestamp_raw:        1723118204.9229074
    completed_timestamp_raw:  1723118268.854065

In [38]:
xdata = dataset.to_xarray_dataarray_dict()

In [43]:
xdata.keys()

dict_keys(['dac_ch2', 'dmm_v1'])

In [44]:
xdata

{'dac_ch2': <xarray.DataArray 'dac_ch2' (index: 10)> Size: 80B
 array([ 0.        ,  2.77777778,  5.55555556,  8.33333333, 11.11111111,
        13.88888889, 16.66666667, 19.44444444, 22.22222222, 25.        ])
 Coordinates:
   * index    (index) int64 80B 0 1 2 3 4 5 6 7 8 9
 Attributes: (12/22)
     name:                     dac_ch2
     paramtype:                numeric
     label:                    Gate ch2
     unit:                     V
     inferred_from:            []
     depends_on:               []
     ...                       ...
     captured_counter:         6
     run_id:                   8
     run_description:          {"version": 3, "interdependencies": {"paramspec...
     parent_dataset_links:     []
     run_timestamp_raw:        1759322796.7466168
     completed_timestamp_raw:  1759322796.7664695,
 'dmm_v1': <xarray.DataArray 'dmm_v1' (dac_ch1: 10)> Size: 80B
 array([4.93766191, 3.0318639 , 1.81055719, 1.00591089, 0.48152592,
        0.14813496, 0.45893428, 0.0

In [42]:
xdata['coords']

KeyError: 'coords'

In [36]:
xdata = dataset.to_xarray_dataset()

/home/nixos/git-repos/arbok_inspector/.venv/lib/python3.12/site-packages/qcodes/dataset/exporters/export_to_xarray.py:191: UserWarning: Independent parameter setpoints are not equal. Check concatenated output carefully. Please consider using `to_xarray_dataarray_dict` to export each independent parameter to its own datarray.
  warnings.warn(


In [37]:
xdata

<xarray.Dataset> Size: 320B
Dimensions:  (index: 10, dac_ch1: 10)
Coordinates:
  * index    (index) int64 80B 0 1 2 3 4 5 6 7 8 9
  * dac_ch1  (dac_ch1) float64 80B 0.0 2.778 5.556 8.333 ... 19.44 22.22 25.0
Data variables:
    dac_ch2  (index) float64 80B 0.0 2.778 5.556 8.333 ... 19.44 22.22 25.0
    dmm_v1   (dac_ch1) float64 80B 4.938 3.032 1.811 ... 0.06775 0.0154 0.1325
Attributes: (12/14)
    ds_name:                  context_example
    sample_name:              synthetic data
    exp_name:                 tutorial_exp
    snapshot:                 {"parameters": {"ch1": {"__class__": "qcodes.pa...
    guid:                     1c37b00d-0000-0000-0000-01999fcf729c
    run_timestamp:            2025-10-01 12:46:36
    ...                       ...
    captured_counter:         6
    run_id:                   8
    run_description:          {"version": 3, "interdependencies": {"paramspec...
    parent_dataset_links:     []
    run_timestamp_raw:        1759322796.7466168
    completed_timestamp_raw:  1759322796.7664695